# Multipole

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook werden die Eigenschaften elektrischer Multipole numerisch untersucht, indem die Multipole durch eine lineare Anordnung von nahe beieinander liegenden elektrischen Ladungen $q_n$ and den Orten $\boldsymbol{r}_n$ dargestellt werden.

Das zugehörige elektrische Feld lässt sich dann aus den Beiträgen der elektrischen Felder der entsprechenden Punktladungen berechnen. In skalierten Variablen ergibt sich

$$\boldsymbol{E}(\boldsymbol{r}) = \sum\limits_n q_n\frac{\boldsymbol{r}-\boldsymbol{r}_n}{|\boldsymbol{r}-\boldsymbol{r}_n|^3}$$

und entsprechend für das elektrische Potential

$$\Phi(\boldsymbol{r}) = \sum\limits_n \frac{q_n}{|\boldsymbol{r}-\boldsymbol{r}_n|}\,.$$

## Importanweisungen

In diesem Jupyter-Notebook verwenden wir zum ersten Mal das SciPy-Modul `special`, das die Berechnung spezieller Funktionen, hier konkret des Binomialkoeffizienten, ermöglicht. 

In [ ]:
from math import cos, pi, sin
import numpy as np
import numpy.linalg as LA
from scipy import integrate, special
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Berechnung der Abstandsabhängigkeit des elektrischen Felds eines Multipols

### Aufbau des Multipols

Um einen Multipol der Ordnung `order` aufzubauen, werden `order+1` Ladungen äquidistant entlang der $z$-Achse angeordnet. Aufgrund der Rotationssymmetrie des Problems genügt es, die Rechnung auf die $x$-$z$-Ebene zu beschränken, wobei die $x$-Koordinaten aller Ladungen gleich null sind. Die Beträge der Ladungen sind durch Binomialkoeffizienten gegeben. In der vorletzten Zeile des Codes wird dafür gesorgt, dass die Vorzeichen der Ladungen alternieren.

In [ ]:
def multipole(order):
    dists = np.arange(-order/2, order/2+1)[:, np.newaxis]
    r_multipole = np.array([[0, 1]]) * dists
    q_multipole = special.comb(order, np.arange(order+1))
    q_multipole[1::2] = -q_multipole[1::2]
    return r_multipole, q_multipole

### Elektrisches Feld des Multipols

Unter Verwendung der Orte `r_multipole` und der Ladungsstärken `q_multipole` der einzelnen Ladungen, die sich mit Hilfe der Funktion `multipole` berechnen lassen, wird unter Verwendung der eingangs angegebenen Formel das elektrische Feld berechnet.

Eine zusätzliche Achse wird hier benötigt, um mittels Broadcasting die korrekte Summation über die vektoriellen Beiträgen ausführen zu können.

In [ ]:
def e_field(r, r_multipole, q_multipole):
    q_multipole = q_multipole[:, np.newaxis]
    distance = LA.norm(r-r_multipole, axis=1)[:, np.newaxis]
    e = np.sum(q_multipole
               * (r-r_multipole)/distance**3, axis=0)
    return e

### Berechnung der Abstandsabhängigkeit des Betrags der elektrischen Feldstärke

Es wird der Betrag des elektrischen Feldes als Funktion des Abstands vom Ursprung unter einem Winkel `theta` zur $z$-Achse berechnet. Da ein Potenzgesetz zu erwarten ist, werden die Abstände zwischen `r_min` und `r_max` äquidistant auf einer logarithmischen Skala gewählt.

Mit Hilfe der Funktion `multipole` wird zunächst die Zerlegung des Multipols der vorgegebenen Ordnung in Einzelladungen bestimmt, um dann mit Hilfe der Funktion `e_field` das zugehörige elektrische Feld zu berechnen. Die Ergebnisse werden in einer Liste `e_values_one_order` gesammelt. Die Listen zu verschiedenen Ordnungen werden dann in der Liste `e_values` zusammengefasst und zusammen mit der Liste der verwendeten Abstände `r_values` zurückgegeben.

In [ ]:
def e_of_r(order_max, theta, r_min, r_max, n_r_max):
    e_values = []
    r_factor = (r_max/r_min)**(1/n_r_max)
    r_values = r_min*r_factor**np.arange(n_r_max)
    direction = np.array([sin(theta), cos(theta)])

    for order in range(order_max+1):
        r_multipole, q_multipole = multipole(order)
        e_values_one_order = []
        for r in r_values:
            x = r * direction
            e = e_field(x, r_multipole, q_multipole)
            e_values_one_order.append(LA.norm(e))
        e_values.append(e_values_one_order)
    return r_values, e_values

### Implementierung der Bedienelemente und graphische Darstellung der Abstandsabhängigkeit des elektrischen Felds

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `order_max`: maximale Ordnung des Multipols
- `theta`: Winkel relativ zur $z$-Achse
- `r_min`: minimaler dimensionsloser Abstand
- `r_max`: maximaler dimensionsloser Abstand

Die Zahl der berechnenden Abstände ist in der Funktion `plot_falloff` mit Hilfe der Variablen `n_r_max` vorgegeben. Bei Bedarf kann diese angepasst werden.

Im Hinblick auf das erwartete Potenzgesetz wird die Abstandsabhängigkeit des Betrags der elektrischen Feldstärke doppelt-logarithmisch dargestellt.

In [ ]:
widget_dict = {"order_max":
               widgets.IntSlider(
                   value=2, min=0, max=10, step=1,
                   description=r"$n_\text{order max}$"),
               "theta":
               widgets.FloatSlider(
                   value=1, min=0, max=pi, step=0.01,
                   description=r"$\theta$"),
               "r_min":
               widgets.FloatLogSlider(
                   value=0.1, min=-2, max=3, step=1,
                   description=r"$r_{\text{min}}/\Delta$"),
               "r_max":
               widgets.FloatLogSlider(
                   value=1e4, min=0, max=5, step=1,
                   description=r"$r_{\text{max}}/\Delta$")
               }

@interact(**widget_dict)
def plot_falloff(order_max, theta, r_min, r_max):
    n_r_max = 1000
    r_values, e_values = e_of_r(order_max, theta, r_min,
                                r_max, n_r_max)

    fig, ax = plt.subplots()
    for order in range(order_max+1):
        ax.loglog(r_values, e_values[order],
                  label=f"ℓ = {order}")
    ax.set_xlabel(r"$r/\Delta$")
    ax.set_ylabel("$E$")
    ax.legend(loc="upper left",
              bbox_to_anchor=(1.05, 1, 1.1, 0))

## Elektrische Feldlinien eines Multipols

Die Feldlinien ergeben sich dadurch, dass das elektrische Feld an jedem Punkt der Feldlinie tangential ist, so dass die Feldlinie durch die Differentialgleichung

$$\frac{\text{d}\boldsymbol{r}}{\text{d}s} = \frac{\boldsymbol{E}(\boldsymbol{r})}{\vert\boldsymbol{E}(\boldsymbol{r})\vert}$$

gegeben ist.

### Implementierung der Differentialgleichung für die Feldlinien

Die Funktion `dr_dt` implementiert die Differentialgleichung für die Feldlinie. Das Argument `direction` kann die Werte ±1 annehmen und gibt die Orientierung an, in der die Feldlinie durchlaufen wird.

In [ ]:
def dr_dt(t, r, r_multipole, q_multipole, direction, eps):
    e = e_field(r, r_multipole, q_multipole)
    return direction * e / LA.norm(e)

Die Berechnung der Feldlinie soll abgebrochen werden, wenn der Abstand zum Ursprung kleiner als das Argument `eps` wird.

In [ ]:
def line_closed(t, r, r_multipole, q_multipole, direction,
                eps):
    return LA.norm(r) - eps

line_closed.terminal = True
line_closed.direction = -1

### Berechnung einer einzelnen Feldlinie

Ausgehend vom Punkt `r_0` in der $x$-$z$-Ebene wird eine Feldlinie berechnet bis entweder die maximale Bogenlänge `t_end` erreicht wurde oder die Abbruchbedingung `line_closed` erfüllt ist. Um die Feldlinie in beiden Richtungen zu erfassen, wird eine Schleife über die Richtung `dir` mit den Werten ±1 ausgeführt. Für jedes Teilstück der Feldlinie werden am Ende `n_max` Datenpunkte berechnet. 

In [ ]:
def one_field_line(t_end, n_max, r_0, eps,
                   r_multipole, q_multipole):
    field_lines = []
    for dir in (1, -1):
        solution = integrate.solve_ivp(
            dr_dt, (0, t_end), r_0,
            args=(r_multipole, q_multipole, dir, eps),
            events=line_closed, dense_output=True,
            atol=1e-10, rtol=1e-10)
        if solution.t_events[0].size > 0:
            t_end = solution.t_events[0][0]
        t_values = np.linspace(0, t_end, n_max)
        field_lines.append(solution.sol(t_values))
    return field_lines

### Berechnung aller Feldlinien

Zur Berechnung der Feldlinien für eine gegebene Ordnung `order` werden zunächst die Ladungen `q_multipole` und Orte `r_multipole` der Punktladungen bestimmt, aus denen der Multipol aufgebaut ist. Anschließend werden für gleichmäßig auf einem Kreis mit Radius `x_start` um den Ursprung angeordnete Startpunkte mit Hilfe von `one_field_line` die zugehörigen zwei Feldlinienstücke berechnet. Die Koordinaten der Feldlinienstücke werden in einer einzigen Liste `field_lines` zusammengefasst.

In [ ]:
def all_field_lines(t_end, n_max, eps, x_start,
                    n_theta_max, order):
    r_multipole, q_multipole = multipole(order)
    field_lines = []
    for theta in np.linspace(0, 2*pi, n_theta_max,
                             endpoint=False):
        r_0 = x_start * np.array([sin(theta), cos(theta)])
        field_line = one_field_line(
            t_end, n_max, r_0, eps, r_multipole,
            q_multipole)
        field_lines.extend(field_line)
    return field_lines

### Implementierung der Bedienelemente und graphische Darstellung der Feldlinien

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `order`: Ordnung des Multipols
- `t_end`: maximale Bogenlänge in einer Richtung
- `x_start`: Abstand des Startpunkts vom Ursprung
- `n_max`: Anzahl der Datenpunkte in jedem Zweig der Feldlinie
- `n_theta_max`: Anzahl der zu betrachtenden Richtungen, unter denen die Feldlinien starten

In [ ]:
widget_dict = {"order":
               widgets.IntSlider(
                   value=2, min=0, max=6, step=1,
                   description=r"$n_\text{order}$"),
               "t_end":
               widgets.FloatSlider(
                   value=1000, min=100, max=2500, step=100,
                   description=r"$t_\text{end}$"),
               "x_start":
               widgets.FloatSlider(
                   value=50, min=10, max=100, step=1,
                   description=r"$x_\text{start}$"),
               "n_max":
               widgets.IntSlider(
                   value=500, min=0, max=1000, step=100,
                   description=r"$n_\text{max}$"),
               "n_theta_max":
               widgets.IntSlider(
                   value=50, min=6, max=100, step=1,
                   description=r"$n_{\theta,\text{max}}$"),
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_field(order, t_end, x_start, n_max, n_theta_max):
    eps = order/2 + 0.1
    field_lines = all_field_lines(
        t_end, n_max, eps, x_start, n_theta_max, order)

    fig, ax = plt.subplots()
    ax.set_aspect("equal")
    for field_line in field_lines:
        ax.plot(field_line[0], field_line[1], color="black")
    ax.set_xlabel("$x$")
    ax.set_xlim(-2*x_start, 2*x_start)
    ax.set_ylabel("$z$")
    ax.set_ylim(-2*x_start, 2*x_start)

## Elektrisches Potential eines Multipols

### Berechnung des elektrischen Potentials

Das Potential wird auf einem zweidimensionalen Gitter mit $2n_\text{max}\times2n_\text{max}$ Punkten berechnet. Das Array `distance` enthält die Abstände zwischen den Gitterpunkten und den Einzelladungen, die den Multipol aufbauen. Daraus ergibt sich das Potential im Array `v` durch Summation entsprechend der eingangs angebenen Formel.

In der vorletzten Zeile wird das Potential auf das Intervall zwischen $-\pi/2$ und $\pi/2$ abgebildet, wobei das Potential null erhalten bleibt. Der Parameter `alpha` dient dazu, die Empfindlichkeit der Farbskala um den Nullpunkt herum einzustellen.

In [ ]:
def potential(order, x_max, n_max, alpha):
    r_multipole, q_multipole = multipole(order)
    r_grid = np.mgrid[-x_max:x_max:2j*n_max,
                      -x_max:x_max:2j*n_max]
    x_grid, z_grid = r_grid
    r_grid = np.moveaxis(r_grid, 0, -1)[:, :, np.newaxis, :]
    distance = LA.norm(r_grid-r_multipole, axis=-1)
    v = np.sum(q_multipole/distance, axis=-1)
    v = np.arctan(alpha*v)
    return x_grid, z_grid, v

### Implementierung der Bedienelemente und graphische Darstellung des elektrischen Potentials

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `order`: Ordnung des Multipols
- `x_max`: halbe Seitenlänge
- `n_max`: halbe Zahl der Punkte je Dimension
- `alpha`: Parameter für die Farbskalierung

In der zweidimensionalen Darstellung wird der Wert des Potentials mittels der rechts gezeigten Farbskala repräsentiert.

In [ ]:
widget_dict = {"order":
               widgets.IntSlider(
                   value=2, min=0, max=6, step=1,
                   description=r"$n_\text{order}$"),
               "x_max":
               widgets.FloatSlider(
                   value=2, min=1, max=50, step=1,
                   description=r"$x_\text{max}$"),
               "n_max":
               widgets.IntSlider(
                   value=100, min=0, max=500, step=10,
                   description=r"$n_\text{max}$"),
               "alpha":
               widgets.FloatLogSlider(
                   value=1, min=0, max=5, step=0.1,
                   description=r"$\alpha$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_potential(order, x_max, n_max, alpha):
    x_grid, z_grid, v = potential(order, x_max, n_max,
                                  alpha)

    fig, ax = plt.subplots()
    ax.set_aspect("equal")
    mesh = ax.pcolormesh(x_grid, z_grid, v, shading="auto")
    ax.set_xlabel("$x$")
    ax.set_ylabel("$z$")
    fig.colorbar(mesh)